# Guide for the compilation of the INCA Core and the implementation of a simple hydrological model

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Cras vulputate, odio quis aliquam pretium, dolor sapien vestibulum mauris, sit amet varius quam metus vitae leo. Nunc nec molestie elit. Nullam lacinia convallis condimentum. Pellentesque nec ante nisi. Cras bibendum mauris quis nibh congue interdum. Suspendisse pharetra augue ac sem ultricies blandit. Aliquam erat volutpat. Fusce nec massa mi.

Proin pretium quam quam, auctor euismod mauris blandit venenatis. Aliquam sed malesuada ex. Integer pellentesque aliquet nunc ut sodales. Integer consectetur libero tellus, vel accumsan eros convallis eu. Morbi porta ultrices mollis. Phasellus orci ligula, vulputate quis accumsan vitae, euismod et lorem. Donec eu dictum est. Interdum et malesuada fames ac ante ipsum primis in faucibus. Cras congue maximus faucibus. Maecenas scelerisque hendrerit urna, a imperdiet dui viverra et. Donec dolor mauris, pulvinar vel interdum et, molestie eget magna. Aliquam in ornare ligula.

In pellentesque mi nec lectus ultrices, luctus fermentum sem convallis. Curabitur sodales gravida elit in facilisis. Cras pretium felis metus, sit amet iaculis lorem pretium vel. Etiam id sodales augue. Aenean congue justo quis fringilla aliquet. Aenean commodo placerat sodales. Cras pharetra orci a ipsum finibus vehicula. Integer velit lorem, vulputate eu augue sit amet, finibus suscipit libero.

Aliquam urna nisl, facilisis at lacinia in, venenatis ac metus. Quisque facilisis, justo id porttitor tincidunt, sapien eros rutrum eros, quis egestas neque felis ut libero. Nullam maximus scelerisque leo non imperdiet. Praesent non semper risus, sed aliquet elit. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Nunc urna magna, elementum varius nunc sit amet, aliquam interdum magna. Duis blandit sed lectus in vehicula. Donec efficitur dapibus turpis, eu volutpat metus interdum condimentum. Nunc purus justo, fringilla ut pharetra in, elementum sit amet justo. Nulla commodo consequat quam, ut tincidunt lectus pulvinar quis. Quisque et metus eget velit aliquam mattis ac vel odio.

In commodo sed libero consectetur hendrerit. Fusce erat sapien, elementum in diam nec, tempus sagittis magna. Cras a ultricies ipsum. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia Curae; Curabitur porta sagittis velit, ut tristique mi laoreet at. Nam posuere libero in mi fermentum, lobortis porta nunc porta. Nulla aliquet luctus ultrices. Donec magna erat, finibus ut cursus nec, placerat a est. In accumsan sem sit amet eros eleifend, at auctor quam tempor. Curabitur tincidunt pulvinar nunc nec pharetra. Maecenas fringilla sem et magna faucibus blandit. Maecenas vulputate nec arcu vel interdum. Nulla hendrerit sem in diam tempus porttitor. Sed maximus, arcu et luctus mattis, justo nunc pretium arcu, at convallis velit est sit amet velit. Vivamus condimentum vulputate faucibus. Proin sodales eros ut consequat tempus.



## Words to the wise



## Python settings

* Importing modules required to run this notebook
* Defining functions used in this notebook
* Setting plotting options


In [4]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib as plt